## Difference Project

Some useful utility function.

In [1]:
function SoftThreshold(z, lambda)
    abs(z) < lambda ? 0. : z > 0 ? z - lambda : z + lambda
end

@show SoftThreshold(3, 1)
@show SoftThreshold(-3, 1)
@show SoftThreshold(-0.5, 1);

SoftThreshold(3,1) => 2
SoftThreshold(-3,1) => -2
SoftThreshold(-0.5,1) => 0.0


Generate data and compute precision matrices

In [2]:
p = 30
Sigmax = eye(p,p)
Sigmay = eye(p,p)
Sigmay[1,2] = 0.5
Sigmay[2,1] = 0.5

sqmy = sqrtm(Sigmay)

n = 1000
X = randn(n,p)
Y = randn(n,p) * sqmy;

hSx = cov(X, corrected=false)
hSy = cov(Y, corrected=false);

Solve the problem using Convex and Mosek for comparison purposes

In [3]:
using Convex
using Mosek

solver = MosekSolver(LOG=1)
set_default_solver(solver);

In [4]:
lambda = 0.3

Delta = Variable(p,p);
prob = minimize(quad_form(vec(Delta), kron(hSy,hSx)) / 2 - trace((hSy-hSx)*Delta) +  lambda * norm(vec(Delta), 1))
prob.constraints += [Delta == Delta']
@time solve!(prob)

sol = Delta.value

Computer
  Platform               : Linux/64-X86    
  Cores                  : 4               

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3606            
  Cones                  : 2               
  Scalar variables       : 2707            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Conic interior-point optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Total number of eliminations : 901
Eliminator terminated.
Eliminator started.
Total number of eliminations : 901
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Eliminator - elim's                 : 901             
Lin. dep.  - tries                  : 1                 time    

30x30 Array{Float64,2}:
 4.81416e-9  0.158126    2.54665e-7  …  1.37762e-7  1.9152e-7   2.72191e-7
 0.158126    1.02049e-7  1.73847e-7     1.14928e-7  2.2328e-7   3.31723e-7
 2.54674e-7  1.73865e-7  1.91358e-7     2.77037e-7  2.41724e-7  1.61299e-7
 2.65504e-7  2.11465e-7  2.49873e-7     2.32698e-7  2.53962e-7  2.0727e-7 
 2.0603e-7   1.30578e-7  2.96083e-7     2.14517e-7  1.9217e-7   2.2986e-7 
 1.56253e-7  1.73598e-7  1.92069e-7  …  1.91077e-7  2.49626e-7  2.67275e-7
 3.06321e-8  9.19215e-8  1.83239e-7     1.94188e-7  1.96385e-7  1.41606e-7
 5.28373e-8  1.86628e-7  2.26729e-7     2.67707e-7  1.33562e-7  1.90318e-7
 3.83402e-7  2.52167e-7  2.09182e-7     2.75221e-7  5.14451e-8  2.28313e-7
 6.41679e-8  1.64049e-7  2.30373e-7     1.74691e-7  1.50785e-7  1.70888e-7
 2.72749e-7  2.11946e-7  3.28234e-7  …  1.84733e-7  2.32469e-7  1.96371e-7
 1.75687e-7  1.66702e-7  3.38736e-7     2.72645e-7  1.54379e-7  2.24946e-7
 3.28179e-7  2.62295e-7  2.02697e-7     2.81869e-7  1.81969e-7  1.02794e-7
 

Shooting algorithm for the objective

In [5]:
function DiffCovShooting(hSx, hSy, lambda, Ups, options)    
    # process options    
    maxIter = 2000
    optTol = 1e-5
    
    #
    p = size(hSx, 1)
        
    Delta = zeros(p, p)
    A = zeros(p, p)
    
    iter = 1;
    while iter < maxIter    
        if mod(iter, 10) == 0
            @show "outer $iter"
        end
    
        fDone = true
        for a=1:p
            for b=a:p
                if a==b
                    # diagonal elements
                    x0 = hSx[a,a]*hSy[a,a]/2
                    x1 = A[a,a] - hSy[a,a] + hSx[a,a]
                    tmp = SoftThreshold(-x1/x0/2 + Delta[a,b], lambda*Ups[a,b] / 2 / x0)
                else
                    # off-diagonal elements
                    x0 = (hSx[a,a]*hSy[b,b] + hSx[b,b]*hSy[a,a])/2 + hSx[a,b]*hSy[a,b]
                    x1 = (A[a,b] + A[b,a])/2 - 2*(hSy[a,b] - hSx[a,b])
                    tmp = SoftThreshold(-x1/x0/2 + Delta[a,b], lambda*Ups[a,b] / x0)
                end
                
                h = tmp - Delta[a,b]
                Delta[a,b] = tmp
                Delta[b,a] = tmp
                if abs(h) > optTol
                    fDone = false
                end
                for j=1:p
                    for k=1:p
                        if a == b
                            A[j,k] = A[j,k] + h * hSx[j,a]*hSy[a,k]
                        else                            
                            A[j,k] = A[j,k] + h * (hSx[j,a]*hSy[b,k] + hSx[j,b]*hSy[a,k])
                        end
                    end
                end
            end
        end
        
        iter = iter + 1;
        if fDone
            break
        end
    end
    Delta
end

DiffCovShooting (generic function with 1 method)

In [6]:
@time solShoot = DiffCovShooting(hSx, hSy, lambda, ones(p,p), 1)

"outer $(iter)" => "outer 10"
elapsed time: 0.080287894 seconds (1780780 bytes allocated)


30x30 Array{Float64,2}:
 0.0       0.316155  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.316155  0.0       0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0       0.0       0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0       0.0       0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0       0.0       0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0       0.0       0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0       0.0       0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0       0.0       0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0       0.0       0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0       0.0       0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0       0.0       0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0       0.0       0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0       0.0       0.0  0.0  0.0  0.0 